In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入mnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
# direct use real_label,normalize var as 0.5
fake_noise_normal = torch.from_numpy(np.random.normal(loc=0.5, scale=0.5, size=[2048,1,32,32])) # var is 1/4
fake_noise_normal = fake_noise_normal.type(torch.FloatTensor)
fake_noise_normal = fake_noise_normal.to(device)
fake_label = torch.from_numpy(np.random.randint(10,size=[2048]))

fake_noise_uniform = np.sqrt(3)*torch.from_numpy(np.random.uniform(0,1,[2048,1,32,32])) # 1/12
fake_noise_uniform = fake_noise_uniform.type(torch.FloatTensor)
fake_noise_uniform = fake_noise_uniform.to(device)

fake_noise_cauchy = torch.from_numpy(np.random.standard_cauchy(size=[2048,1,32,32])+0.5)
fake_noise_cauchy = fake_noise_cauchy.type(torch.FloatTensor)
fake_noise_cauchy = fake_noise_cauchy.to(device)

fake_noise_laplace = torch.from_numpy(np.random.laplace(loc=0.5, scale=1.0, size=[2048,1,32,32]))/ np.sqrt(8) # var is 2
fake_noise_laplace = fake_noise_laplace.type(torch.FloatTensor)
fake_noise_laplace = fake_noise_laplace.to(device)

fake_noise_gumbel = torch.from_numpy(np.random.gumbel(loc=0.5,scale=1.0,size=[2048,1,32,32]))/ np.sqrt(6.58) #pi^2/6 1.645
fake_noise_gumbel = fake_noise_gumbel.type(torch.FloatTensor)
fake_noise_gumbel = fake_noise_gumbel.to(device)
# from 0.01 - 0.99,0 -1
epsilon_noise = 0.1

In [4]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [5]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10



In [6]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius_noise(radius1,radius2,netD,noise, real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [7]:
# load_model
netD = torch.load('MNIST_NetD-mode11-radius2.pth') 
netV = torch.load('MNIST_NetV-mode11-radius2.pth') 
netG = torch.load('MNIST_NetG-mode11-radius2.pth')

mode = 11
gam3 = 1
net_noise1 = get_noise(DATASET, device, LABEL)

In [8]:
#sample the length of radius,gaussian noise
mode = 11
gam3 = 1
L_noise= []
epsilon = 0.05
L_noise1= []
epsilon1 = 0.1

L_noise2= []
epsilon2 = 0.2

L_noise3= []
epsilon3 = 0.3
L_noise4= []
epsilon4 = 0.4
L_noise5= []
epsilon5 = 0.5
L_noise6= []
epsilon6 = 0.6
L_noise7= []
epsilon7 = 0.7
L_noise8= []
epsilon8 = 0.8
L_noise9= []
epsilon9 = 0.9

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise.append(r1)
print(max(L_noise),min(L_noise),np.mean(L_noise),np.var(L_noise))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise1.append(r1)
print(max(L_noise1),min(L_noise1),np.mean(L_noise1),np.var(L_noise1))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise2.append(r1)
print(max(L_noise2),min(L_noise2),np.mean(L_noise2),np.var(L_noise2))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise3.append(r1)
print(max(L_noise3),min(L_noise3),np.mean(L_noise3),np.var(L_noise3))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise4.append(r1)
print(max(L_noise4),min(L_noise4),np.mean(L_noise4),np.var(L_noise4))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise5.append(r1)
print(max(L_noise5),min(L_noise5),np.mean(L_noise5),np.var(L_noise5))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise6.append(r1)
print(max(L_noise6),min(L_noise6),np.mean(L_noise6),np.var(L_noise6))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise7.append(r1)
print(max(L_noise7),min(L_noise7),np.mean(L_noise7),np.var(L_noise7))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise8.append(r1)
print(max(L_noise8),min(L_noise8),np.mean(L_noise8),np.var(L_noise8))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise9.append(r1)
print(max(L_noise9),min(L_noise9),np.mean(L_noise9),np.var(L_noise9))

0.5057213738948494
0.5053531606954061
0.5088396979481444
0.4956299019409689
0.5138871545167579
0.48649677127480184
0.503198051846385
0.5195289191193343
0.5276996474143819
0.5072805951719055
0.5129237474782832
0.4899869416068444
0.4843420459707275
0.540316568044471
0.5129237474782832
0.47269046721980584
0.4981522200449367
0.4937011531703747
0.4962252874907463
0.4855329355475232
0.540316568044471 0.47269046721980584 0.5030215193937465 0.0002440230322165709
0.5104006172258705
0.5047579668563746
0.49718869452922104
0.5012708090806316
0.48085562715708113
0.5043889414231807
0.5302225407123126
0.5302225407123126
0.5210880218955734
0.4991158187941573
0.4981522200449367
0.5012708090806316
0.4777359888096912
0.5094377121478536
0.4849375499977458
0.5078760991989238
0.5057213738948494
0.49154616288390046
0.5144829687553392
0.4752129770345196
0.5302225407123126 0.4752129770345196 0.5022942720117554 0.0002250083279664953
0.4783331173423674
0.45345924618376704
0.5022346448079102
0.49466456020884947
0

In [9]:
mode = 11
gam3 = 1

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.49154635459464635
0.4911781413952031
0.4884280305413513
0.4808554354463352
0.4697988134710811
0.4818190341955559
0.5695061081658215
0.8651669106550752
2.3457638481517193
3.610463403065011
3.3546853080986314


In [10]:
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

3.089506987402706
4.6742268740151625
4.844682966252315
4.913915473130576
4.95667200218944
4.908274329012856
4.923411367785785
4.9419008582157335
4.974928944292159
5.015168723642975


In [11]:
#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.49214174014442374
0.48998725181840735
0.48301042573280945
0.47989190993061953
0.5129236290010422
0.6308923565787021
1.0101720055302037
2.0835299401414886
2.395202283082374
2.378796187706998


In [12]:
#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.4911781413952031
0.4884280305413513
0.47521347899497746
0.4594694157346817
0.46102832680017475
0.5179700080538412
0.7658347132193905
1.9863446532011184
3.8451210621370095
4.115646233547853


In [13]:
#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.48998725181840735
0.4884280305413513
0.47521347899497746
0.46451455386492624
0.48048609966354294
0.5648287997753794
0.837543667581139
3.2924658667543647
4.20347666676743
3.8162492521665974


In [14]:
mode

11

In [15]:
# set mode as 1, test again
mode = 1
gam3 = 1

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#cauchy
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

4.101245429463065
4.127301874028734
4.167799921539784
4.282647577428476
4.458764460880661
4.8578993346380654
5.62425264807115
6.784446709192351
7.640455481203716
7.819103501783684
7.764668433857612
99.88242207034902
97.6802976623264
99.49620706448802
99.85191448572245
99.96096336867856
99.92169612408492
99.91857717950862
99.92325510838391
99.78019652107302
99.79653415243529
4.15903450146605
4.227865186101017
4.460323563680476
4.89404636529079
5.5572183723655595
6.401629007551459
7.045935987444671
7.622195910089643
99.98211347073463
99.89274346174224
4.110968614945848
4.115647547841948
4.1515709206610945
4.185094077288661
4.353051251645116
4.906107891475957
6.874474753634528
12.007961377424216
10.298199956080746
8.875231316556276
4.1235881644256605
4.148345472290689
4.242413986958435
4.426326813065921
4.827386683975593
5.563222161324951
7.124255252446309
8.228867802779941
8.46602387787207
8.352093329627266


In [16]:
# set mode as 1, test again
mode = 1
gam3 = 0.2

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#cauchy
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.1307892315631369
0
0
0
0
0
3.900824266690192
5.2924269124549745
6.141369515588715
6.0000865671034465
5.742832656454214
8.204416871623819
8.631078378619327
8.736195271432473
8.985288659805864
9.15069122483283
9.07059414647132
9.095088165492887
8.978085163777116
8.998499149622276
9.004140914224854
0
0
0
0
0.6774102819776351
4.690924831818883
5.696211983134144
5.677725313302813
5.682405250305039
5.406915365431881
0
0
0
0
0
0.7123700560475619
4.072847275251648
5.184559908787976
6.024358124022985
5.938818151326799
0
0
0
0
0
3.687851676033361
4.8928589169590975
5.961385237794175
6.177316021271907
6.154213918757125


In [17]:
# set mode as 1, test again
mode = 1
gam3 = 0.1

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#cauchy
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0
0
0
0
0
0
0
4.307857872305416
5.3083889460054765
5.36437692516404
5.028761431103903
6.057027007187625
6.520800960239197
6.763063127466086
6.750197524135422
6.847614978929662
6.813024850977563
6.78093677181623
6.7743235216682915
6.756434792765016
6.778180280370856
0
0
0
0
0
0.05596570458118357
3.975660363682147
4.296464826697128
4.077889976081286
3.8533904292408865
0
0
0
0
0
0
0
4.199620720460246
5.005826323147828
5.297104147573763
0
0
0
0
0
0.022337167189647757
3.960289269992323
5.052299190882892
5.636499037039851
5.449710068979626


In [18]:
# set mode as 1, test again
mode = 1
gam3 = 10

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#cauchy
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

4.911381324378272
4.905145296671232
4.9447903854366135
5.017109113367308
5.11342317683068
5.484901460912775
6.303849102104056
8.372554116789322
99.80098416999816
99.95688082535304
99.98330436031141
99.98486358158847
99.90499097329429
99.79304780689328
99.85191448572245
99.96096336867856
99.92169612408492
99.96890267070665
99.92325510838391
99.9541307144992
99.988946124914
4.959195013964611
5.021788046263408
5.204007977445548
5.540883296381568
6.179838531086621
7.095893457154004
7.9752197487199075
99.90885117366517
99.98211347073463
99.9928007569271
4.892854353268681
4.891337870170922
4.886253051646842
4.859827581348315
5.015550010591069
5.741500534255837
99.95376269301049
99.89838491613325
99.69718826175959
99.9277048609644
4.915247093368494
4.947910097055657
4.998239960533071
5.137921404140095
5.450900958556422
6.349141096002109
9.760993074818293
16.2363571090584
99.86356950571846
99.85346438650026


In [19]:
#sample the length of radius,gaussian noise, cauchy noise, uniform noise.
mode = 11
gam3 = 1
L_0= []
L_1= []
L_2= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_0.append(r1)
    
for _ in range(20):
    r1,r2 = gs_radius2(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_1.append(r1)
    
for _ in range(20):
    r1,r2 = gs_radius3(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_2.append(r1)

0.47929652438084214
0.5210885238560312
0.501638830569329
0.47580886454475485
0.4849378602093088
0.5154463757938139
0.5323772660163869
0.5230224122955938
0.5374239100515859
0.5276996474143819
0.5271046453478214
0.4651104413751615
0.4965916844737858
0.48241492170579114
0.5123282434512648
0.5022343345963474
0.5323772660163869
0.5129240576898461
0.5257725231494456
0.48746037002402254
4.312226410541606
0.00504475469974662
10.55936226750394
2.4505330292531675e-06
28.364523036366922
0.001103862094520546
0.04490607013493592
13.911839403307827
0
6.663246985728307e-07
15.227073593513223
24.971390700355453
5.813085195271202e-06
9.136424350014426
16.429330919688578
19.352150350340366
0.0021542966152704674
24.367787973710985
0
19.364992626819298
0.008163653932434496
0.007567647944958251
0.008759039482211878
0.008759039482211878
0.008531602202196918
0.008531602202196918
0.007567647944958251
0.008759039482211878
0.008531602202196918
0.008390425525448808
0.008759039482211878
0.008986431512925054
0.008

In [20]:
L_0

[0.47929652438084214,
 0.5210885238560312,
 0.501638830569329,
 0.47580886454475485,
 0.4849378602093088,
 0.5154463757938139,
 0.5323772660163869,
 0.5230224122955938,
 0.5374239100515859,
 0.5276996474143819,
 0.5271046453478214,
 0.4651104413751615,
 0.4965916844737858,
 0.48241492170579114,
 0.5123282434512648,
 0.5022343345963474,
 0.5323772660163869,
 0.5129240576898461,
 0.5257725231494456,
 0.48746037002402254]

In [21]:
L_1

[4.312226410541606,
 0.00504475469974662,
 10.55936226750394,
 2.4505330292531675e-06,
 28.364523036366922,
 0.001103862094520546,
 0.04490607013493592,
 13.911839403307827,
 0,
 6.663246985728307e-07,
 15.227073593513223,
 24.971390700355453,
 5.813085195271202e-06,
 9.136424350014426,
 16.429330919688578,
 19.352150350340366,
 0.0021542966152704674,
 24.367787973710985,
 0,
 19.364992626819298]

In [22]:
L_2

[0.008163653932434496,
 0.007567647944958251,
 0.008759039482211878,
 0.008759039482211878,
 0.008531602202196918,
 0.008531602202196918,
 0.007567647944958251,
 0.008759039482211878,
 0.008531602202196918,
 0.008390425525448808,
 0.008759039482211878,
 0.008986431512925054,
 0.008163033494735631,
 0.008531602202196918,
 0.007567647944958251,
 0.008759039482211878,
 0.008163033494735631,
 0.007567647944958251,
 0.008531602202196918,
 0.008531602202196918]

In [23]:
#sample the length of radius
L_gaussian= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_gaussian.append(r1)
    
print(max(L_gaussian),min(L_gaussian),np.mean(L_gaussian),np.var(L_gaussian))

3.0974552997184484
2.027541960982926
1.77835534052283
3.2955841908076597
2.5298614638429315
2.2769369167280327
3.330183329047417
2.4653491959680167
2.778454557251316
1.9620294258747497
1.8809664299642288
2.4307610759812164
2.7724833744230333
3.24406738134403
2.5751701648688674
2.7116303920350275
3.7510416452394115
2.182266943776292
2.5497069791635023
3.5723749803600366
3.7510416452394115 1.77835534052283 2.660611052394999 0.31054080059679307


In [24]:
#sample the length of radius
L_cauchy= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_cauchy.append(r1)
    
print(max(L_cauchy),min(L_cauchy),np.mean(L_cauchy),np.var(L_cauchy))

5.103597217884295
5.009890033241004
4.996693812966518
4.3143563030889975
4.187724523692394
5.047614497918273
4.6324059063585015
4.7727321413095005
5.762651950867529
5.8066031212449865
5.2883443691294545
5.021192158753173
4.876157636387514
5.1743424549960055
4.322033083316365
4.4807437383682664
4.852626212009042
5.490910818229939
4.6843187746578465
5.005454860414028
5.8066031212449865 4.187724523692394 4.941519680741682 0.18573087995632395


In [26]:
#sample the length of radius
L_laplace= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_laplace.append(r1)
    
print(max(L_laplace),min(L_laplace),np.mean(L_laplace),np.var(L_laplace))

3.8757352908929565
3.795734382122524
4.956669875499966
4.3566261507765125
3.157510740488582
3.631512071755566
2.923493953804088
2.902112498767297
5.6374658853234285
3.5443923426018435
4.17666790171714
4.488969975552058
3.1987849010019462
3.018143776851586
3.8476186658235005
3.442526068800886
4.144963740831386
4.388015781839064
3.253791068787271
3.258838214882814
5.6374658853234285 2.902112498767297 3.7999786644060194 0.49430342707641606



# other data sets, using same noise to get different r for diff data
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET2 = 'FMNIST'
ROOT2    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader2, dataset2 = get_dataloader(
        data_name=DATASET2,
        root=ROOT2,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader2 = iter(dataloader2)
data2, label2 = next(iter_dataloader2)
data_time = time.time() - data_time
real_cpu2 = data2.to(device)
real_label2 = label2.clone().detach()


device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET3 = 'cifar10'
ROOT3    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader3, dataset3 = get_dataloader(
        data_name=DATASET3,
        root=ROOT3,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader3 = iter(dataloader3)
data3, label3 = next(iter_dataloader3)
data_time = time.time() - data_time
real_cpu3 = data3.to(device)
real_label3 = label3.clone().detach()

sample1 = real_cpu2[:,0,:,:]
sample2 = real_cpu2[:,1,:,:]
sample3 = real_cpu2[:,2,:,:]